In [ ]:
%matplotlib inline

import sqlite3
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from Utility import Database as db
from Utility import Tabulate

In [ ]:
# def delete_results(problem_name, population_size):
#     db.con.execute(f'''
#         DELETE FROM 
#             result 
#         WHERE ROWID IN  (
#             SELECT 
#                 result.ROWID 
#             FROM 
#                 result 
#             INNER JOIN 
#                 config 
#             ON 
#                 (config.config_id = result.config_id) 
#             WHERE 
#                 config.name="{problem_name}" and
#                 config.population_size={population_size}
#         )
#     ''').fetchall()
    
#     db.con.execute(f'''
#         DELETE FROM 
#             config
#         where
#             config.name="{problem_name}" and
#             config.population_size={population_size}
#     ''')
    
# # delete_results('Rosenbrock', 320)
# # db.save_and_quit()

In [ ]:
def plot_epochs_migration_rate_vs_fitness(problem_name, population_size):
    # same y axis
    # get best results for an algorithm table
    algorithms = [
        'Island GA Ring Lattice',
        'Island GA Cell',
        'Island GA Hierarchy',
        'Island GA Caveman',
        'Island GA Rewired Caveman',
        'Island GA Watts Strogatz',
        'Island GA Full'
    ]
    
    fig = plt.figure(figsize=(15,18))
    subplots = fig.subplots((len(algorithms)//2)+1, 2, subplot_kw=dict(projection='3d'),)
    subplots = [item for sublist in subplots for item in sublist]
    
    min_fitness = 10000000000
    max_fitness = -10000000000
    
    for i, alg_name in enumerate(algorithms):
        results = db.con.execute(f'''
            select 
                config.epochs_till_migration,
                config.migration_rate,
                fitness
            from 
                result
            inner join 
                config
            on 
                result.config_id == config.config_id
            where 
                name="{problem_name}" and 
                population_size={population_size} and 
                algorithm="{alg_name}"
        ''').fetchall()

        df = pd.DataFrame(
            results, 
            columns=['Epoch', 'Migration Rate', 'Fitness'])
        df = df.groupby(['Epoch', 'Migration Rate']).mean().reset_index()
        
        ax = subplots[i]
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Migration Rate')
        ax.set_zlabel('Fitness')
        ax.set_title(alg_name)

        ax.plot_trisurf(df['Epoch'], df['Migration Rate'], df['Fitness'])
        
        min_fitness = min(min_fitness, df['Fitness'].min())
        max_fitness = max(max_fitness, df['Fitness'].max())
        
        
    for ax in subplots:
        ax.set_zlim(min_fitness, max_fitness)
    plt.show()

In [ ]:
def get_best_results(problem_name, population_size):
    algorithms = [
        'Island GA Ring Lattice',
        'Island GA Cell',
        'Island GA Hierarchy',
        'Island GA Caveman',
        'Island GA Rewired Caveman',
        'Island GA Watts Strogatz',
        'Island GA Full'
    ]
    
    for i, alg_name in enumerate(algorithms):
        results = db.con.execute(f'''
            select 
                config.algorithm, 
                fitness,
                config.migration_rate,
                config.epochs_till_migration
            from 
                result
            inner join 
                config
            on 
                result.config_id == config.config_id
            where 
                name="{problem_name}" and 
                population_size={population_size}
            group by 
                config.algorithm
            having
                result.fitness == min(result.fitness)
            order by 
                result.fitness
        ''').fetchall()
        
    Tabulate.print_table([
            'Algorith', 
            'Fitness', 
            'Migration Rate', 
            'Epochs Per Migraiton'
        ],
        results)


In [ ]:
def plot_calculations_fitness(problem_name, population_size):    
    results = db.con.execute(f'''
        select 
            config.algorithm, 
            config.fitness_calculations,
            config.epochs_till_migration,
            config.migration_rate,
            fitness
        from 
            result
        inner join 
            config
        on 
            result.config_id == config.config_id
        where 
            name="{problem_name}" and 
            population_size={population_size}
    ''').fetchall()

    df = pd.DataFrame(
        results, 
        columns=['Algorithm', 'Fitness Calculations', 'Epoch', 'Migration Rate', 'Fitness'])
    
#     print(df.to_string(index=False))
    sns.set(rc = {'figure.figsize':(15,8)})
    sns.set_theme(style="darkgrid")
    sns.lineplot(data=df, x='Fitness Calculations', y='Fitness', hue='Algorithm')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
def algorithm_best(problem_name, population_size, limit):
    algorithms = [
        'Island GA Ring Lattice',
        'Island GA Cell',
        'Island GA Hierarchy',
        'Island GA Caveman',
        'Island GA Rewired Caveman',
        'Island GA Watts Strogatz',
        'Island GA Full'
    ]
    
    for i, alg_name in enumerate(algorithms):
        results = db.con.execute(f'''
            select 
                fitness,
                config.migration_rate,
                config.epochs_till_migration
            from 
                result
            inner join 
                config
            on 
                result.config_id == config.config_id
            where 
                name="{problem_name}" and 
                population_size={population_size} and
                algorithm="{alg_name}"
        ''').fetchall()
        
        df = pd.DataFrame(
            results, 
            columns=['Fitness', 'Migration Rate', 'Epoch'])
        df = df.groupby(['Epoch', 'Migration Rate']).mean().reset_index()

        
        print(alg_name)
        print(df.nsmallest(limit, 'Fitness'))
        
        print()

## Binary Alpine, 1280

In [ ]:
# plot_epochs_migration_rate_vs_fitness('Alpine', 640)

In [ ]:
plot_calculations_fitness('Alpine', 1280)

## Rastrigin, 640

In [ ]:
plot_epochs_migration_rate_vs_fitness('Rastrigin', 640)

In [ ]:
algorithm_best('Rastrigin', 640, 10)

In [ ]:
get_best_results('Rastrigin', 640)

## TSP, 320

In [ ]:
plot_epochs_migration_rate_vs_fitness('TSP', 320)

In [ ]:
get_best_results('TSP', 320)

In [ ]:
algorithm_best('TSP', 320, 10)

## Rosenbrock, 320

In [ ]:
plot_calculations_fitness('Rosenbrock', 320)

In [ ]:
algorithm_best('Rosenbrock', 320, 10)